# 類似画像の検出をしよう

## 簡単な形状認識 - Average Hash

最初に、画像の形状を手軽に検出する方法を紹介する。Average Hashを利用すると、画像の類似地を計算することができる。  
Average Hashは、画像を比較可能なハッシュ値で表す。ハッシュ関数MD5やSHA256などを使うと、様々なデータの値を要約したハッシュ値を得ることができる。そして、これらのハッシュ値は同一データを検出するのに非常に役立つ。  
しかし、大きな意味で画像が同一かどうかを検出するのに、MD5やSHA256などのハッシュ関数を利用することはできない。完全に同一のバイナリを探すならば、こちらのハッシュ関数を利用することができるが、画像データは画像解像度をリサイズしたり、色調補正したり、JPEG/PNGと圧縮形式を変更したりと、様々な変更が加えられ、完全に同一ではないことも多い。  
写真を多少明るく補正してあったとしても、同一の画像として検索できた方が便利な場面が多くあり、その時に役立つのが、類似度の計算に使えるAverage Hashである。  
具体的な方法だが、以下のような手順で行う。  
1. 画像のサイズを8*8に縮小
2. 色をグレイスケールに変換
3. 画像の各ピクセルの平均値を計算
4. 各ピクセルの濃淡が平均より大きければ1、平均以下なら0とする  
以上のような簡単な手順で、画像の形状を表す64ビットのハッシュ値が得られる。この方法であれば、64ビットを比較するだけなので、高速に類似画像を検索することができる。

In [3]:
from PIL import Image
import numpy as np

In [7]:
# 画像データをAverage Hashに変換
def average_hash(fname, size = 16):
    img = Image.open(fname)
    img = img.convert('L') # グレースケールに変換
    img = img.resize((size, size), Image.ANTIALIAS) # アンチエイリアス処理を施した後リサイズ
    pixel_data = img.getdata() # ピクセルデータを得る
    pixels = np.array(pixel_data) # Numpyの配列に変換
    pixels = pixels.reshape((size, size)) # 二次元の配列に変換
    avg = pixels.mean() # 算術平均を計算
    diff = 1 * (pixels > avg) # 平均より大きければ値を1、平均以下で0に変換
    return diff

In [8]:
# 二進数とみなしてハッシュ値に変換
def np2hash(n):
    bhash = []
    for nl in ahash.tolist():
        sl = [str(i) for i in nl]
        s2 ="".join(sl)
        i = int(s2, 2) # 二進数を整数に
        bhash.append("%04x" % i)
    return "".join(bhash)

In [9]:
# Average Hashを表示
ahash = average_hash('tower.jpg')
print(ahash)
print(np2hash(ahash))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0]
 [1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0]
 [1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]]
000001000180018003c003c003c087e07ff05ff07ff2cffa9ffe3ffe3ffe1fff


### たくさんの画像から類似する画像を検索する

Average Hashがわかったところで、たくさんの画像の中からこの画像に類似するものを選び出すプログラムを作っていく。ここでは、サンプル画像として、カリフォルニア工科大学が機械学習のために配布している様々な画像データセット「Computational Vision at CALTECH」を利用してみる。

In [27]:
from PIL import Image
import numpy as np
import os, re

In [28]:
# ファイルパスの指定
search_dir = "./image/101_ObjectCategories"
cache_dir = "./image/cache_avhash"

In [29]:
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

In [30]:
# 画像データをAverage hashに変換
def average_hash(fname, size=16):
    fname2 = fname[len(search_dir):]
    # 画像をキャッシュしておく
    cache_file = cache_dir + "/" + fname2.replace('/', '_') + ".csv"
    if not os.path.exists(cache_file): # ハッシュを作成
        img = Image.open(fname)
        img = img.convert('L').resize((size, size), Image.ANTIALIAS)
        pixels = np.array(img.getdata()).reshape((size, size))
        avg = pixels.mean()
        px = 1 * (pixels > avg)
        np.savetxt(cache_file, px, fmt="%.0f", delimiter=",")
    else: # すでにキャッシュがあればファイルから読み込み
        px = np.loadtxt(cache_file, delimiter=",")
    return px

In [31]:
# 簡単にハミング距離を求める
def hamming_dist(a, b):
    aa = a.reshape(1, -1) # 1次元の配列に変換
    ab = b.reshape(1, -1)
    dist = (aa != ab).sum()
    return dist

In [32]:
# 全てのディレクトリを列挙
def enum_all_files(path):
    for root, dirs, files in os.walk(path):
        for f in files:
            fname = os.path.join(root, f)
            if re.search(r'\.(jpg|jpeg|png)$', fname):
                yield fname

In [33]:
# 画像を検索
def find_image(fname, rate):
    src = average_has(fname)
    for fname in enum_all_files(search_dir):
        dst = average_hash(fname)
        diff_r = hamming_dist(src, dst)/256
        # print("[check] ", fname)
        if diff_r < rate:
            yield (diff_r, fname)

In [34]:
# 検索
srcfile = search_dir + "/chair/image_0016.jpg"
html = ""
sim = list(find_image(srcfile, 0.25))
sim = sorted(sim, key=lambda x:x[0])
for r, f in sim:
    print(r, ">", f)
    s = '<div style="float:left;"><h3>[差異:' + str(r) + '-' + \
        os.path.basename(f) + ']</h3>' + \
    '<p><a href="' + f + '"><img src="' + f + '" width = 400>' + \
    '</a></p></div>'
    html += s
# HTMLを出力
html = """<html><body><h3>元画像</h3><p>
<img src='{0}' width=400></p>{1}
</body></html>""".format(srcfile, html)
with open("./avhash-search-output.html", "w", encoding="utf-8") as f:
    f.write(html)
print("ok")

0.0 > ./image/101_ObjectCategories/chair/image_0016.jpg
0.22265625 > ./image/101_ObjectCategories/airplanes/image_0129.jpg
0.2265625 > ./image/101_ObjectCategories/chair/image_0031.jpg
0.2265625 > ./image/101_ObjectCategories/stop_sign/image_0019.jpg
0.234375 > ./image/101_ObjectCategories/umbrella/image_0009.jpg
0.23828125 > ./image/101_ObjectCategories/airplanes/image_0124.jpg
0.24609375 > ./image/101_ObjectCategories/dragonfly/image_0001.jpg
0.24609375 > ./image/101_ObjectCategories/chair/image_0001.jpg
ok


ハミング距離とは、等しい文字数を持つ3つの文字列の中で、対応する一にあることなった文字の個数のこと。ここでは、画像一つを256字のハッシュ値で表しているため、何文字異なるかを調べて、それを画像の差異として表している。

# CNNでCaltech 101の画像分類に挑戦しよう

## CNNでカラー画像の分類に挑戦しよう

Caltech 101には画像が101種類のカテゴリー二分されているが、全部を分類するのは学習に時間がかかるため、ここでは、その中から5種類のデータを学習させて、正しく分類できるかを試してみる。

### 画像データをPythonのデータ型に変換しよう

画像を学習する上で、毎回フルサイズの画像を読み込んで処理するのは効率的ではなく、MNISTの画像セットも同じサイズに成形されておりm画像ファイルではなく、扱いやすいデータ型で配布されていた。  
Caltech 101の画像を眺めてみると、それぞれの画像は不定形であり、機械学習で扱うには、不都合であることがわかる。そこで、画像を一定のサイズにリサイズした上で、画像を24ビットRGB形式に変換しておく。また、機械学習ですぐ使えるように、Numpyの配列型式で保存することにする。

Caltech 101の画像データセットを処理して、「image/5obj.npy」というファイルに保存しよう。

In [9]:
from sklearn.model_selection import train_test_split
from PIL import Image
import os, glob
import numpy as np

In [4]:
# 分類対象のカテゴリーを選ぶ
caltech_dir = "./image/101_ObjectCategories"
categories = ["chair", "camera", "butterfly", "elephant", "flamingo"]
nb_classes = len(categories)

In [5]:
# 画像サイズを指定
image_w = 64
image_h = 64
pixels = image_w * image_h * 3

In [6]:
# 画像データを読み込み
X = []
Y = []
for idx, cat in enumerate(categories):
    # ラベルを指定
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    # 画像
    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir + "/*.jpg")
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        X.append(data)
        Y.append(label)
        if i % 10 == 0:
            print(i, "\n", data)
X = np.array(X)
Y = np.array(Y)

0 
 [[[ 40  24  31]
  [ 33  20  24]
  [ 27  16  20]
  ...
  [ 77  69  93]
  [ 75  66  92]
  [ 70  61  89]]

 [[ 40  25  32]
  [ 34  20  24]
  [ 31  19  23]
  ...
  [ 80  72  96]
  [ 77  68  94]
  [ 75  65  94]]

 [[ 41  26  33]
  [ 35  21  25]
  [ 33  22  26]
  ...
  [ 84  77 100]
  [ 82  74 100]
  [ 78  69  97]]

 ...

 [[189 162 192]
  [189 169 198]
  [175 160 191]
  ...
  [204 173 206]
  [202 170 204]
  [196 164 198]]

 [[154 131 164]
  [141 121 153]
  [137 119 150]
  ...
  [200 170 204]
  [195 165 200]
  [191 161 195]]

 [[129 111 145]
  [165 143 178]
  [184 157 195]
  ...
  [198 168 202]
  [199 169 203]
  [196 166 200]]]
10 
 [[[7 7 7]
  [4 4 4]
  [3 3 3]
  ...
  [4 4 4]
  [4 4 4]
  [8 8 8]]

 [[4 4 4]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [1 1 1]
  [4 4 4]]

 [[4 4 4]
  [1 1 1]
  [0 0 0]
  ...
  [0 0 0]
  [2 2 2]
  [3 3 3]]

 ...

 [[4 4 4]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [2 2 2]
  [3 3 3]]

 [[4 4 4]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [2 2 2]
  [3 3 3]]

 [[7 7 7]
  [4 

30 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 253 255]
  [253 254 253]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [237 236 237]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [196 198 197]
  [240 240 240]
  [254 254 254]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
40 
 [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0

10 
 [[[  8   6   7]
  [  9   8   8]
  [ 34  35  34]
  ...
  [  8   8   6]
  [  8   8   5]
  [  8   9   4]]

 [[  7   9   8]
  [ 14  16  15]
  [ 21  20  21]
  ...
  [  8   8   6]
  [  8   9   5]
  [  8   9   4]]

 [[ 11  18  16]
  [ 10  14  13]
  [  9   6   7]
  ...
  [  8   8   6]
  [  8   9   5]
  [  8   9   4]]

 ...

 [[ 76  94  67]
  [ 81 102  72]
  [ 77  99  70]
  ...
  [ 67  87  62]
  [ 63  83  60]
  [ 66  87  63]]

 [[ 71  91  64]
  [ 70  91  60]
  [ 68  88  59]
  ...
  [ 89 108  84]
  [ 84 105  84]
  [ 84 106  85]]

 [[ 68  88  61]
  [ 68  89  58]
  [ 66  87  58]
  ...
  [ 85 104  80]
  [ 72  93  72]
  [ 81 103  82]]]
20 
 [[[ 91 116 145]
  [ 90 116 143]
  [ 91 114 138]
  ...
  [ 16  14   8]
  [ 12  11   8]
  [  9  10   7]]

 [[ 95 118 151]
  [ 94 116 147]
  [ 92 113 140]
  ...
  [ 10  10   5]
  [  8  10   4]
  [  5   8   2]]

 [[ 94 115 141]
  [ 97 117 141]
  [ 98 116 136]
  ...
  [ 19  19  22]
  [ 18  20  29]
  [ 31  34  47]]

 ...

 [[127 140 127]
  [127 139 127]
  [131 143

In [10]:
# 学習データとテストデータを分ける
X_train, X_test, y_train, y_test = train_test_split(X, Y)
xy = (X_train, X_test, y_train, y_test)
np.save("./image/5obj.npy", xy)
print("ok", len(Y))

ok 334


### CNNを実践してみよう

In [13]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [14]:
# カテゴリーの指定
categories = ["chair", "camera", "butterfly", "elephant", "flamingo"]
np.classes = len(categories)

In [15]:
# 画像サイズを指定
image_w = 64
image_h = 64

In [17]:
# データをロード
X_train, X_test, y_train, y_test = np.load("./image/5obj.npy", allow_pickle=True)

In [18]:
# データを正規化する
X_train = X_train.astype("float")/256
X_test = X_test.astype("float")/256
print("X_train shape:", X_train.shape)

X_train shape: (250, 64, 64, 3)


In [23]:
# モデルを構築
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode="same", input_shape=X_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Convolution2D(64, 3, 3, border_mode="same"))
model.add(Activation("relu"))
model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation("softmax"))

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_5 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 64)       

<ipython-input-23-046af88ff502>:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., padding="same")`
  model.add(Convolution2D(32, 3, 3, border_mode="same", input_shape=X_train.shape[1:]))
<ipython-input-23-046af88ff502>:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
  model.add(Convolution2D(64, 3, 3, border_mode="same"))
<ipython-input-23-046af88ff502>:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  model.add(Convolution2D(64, 3, 3))


In [20]:
# モデルを訓練する
model.fit(X_train, y_train, batch_size=32, nb_epoch=50)

<ipython-input-20-12515c3dfdf4>:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  model.fit(X_train, y_train, batch_size=32, nb_epoch=50)


Epoch 1/50
250/250 [==============================] - 4s 14ms/step - loss: 1.2044 - accuracy: 0.7336
Epoch 2/50
250/250 [==============================] - 2s 9ms/step - loss: 0.4865 - accuracy: 0.7984
Epoch 3/50
250/250 [==============================] - 2s 8ms/step - loss: 0.4687 - accuracy: 0.8016
Epoch 4/50
250/250 [==============================] - 2s 7ms/step - loss: 0.4584 - accuracy: 0.8136
Epoch 5/50
250/250 [==============================] - 2s 7ms/step - loss: 0.4819 - accuracy: 0.8032
Epoch 6/50
250/250 [==============================] - 2s 7ms/step - loss: 0.4284 - accuracy: 0.8144
Epoch 7/50
250/250 [==============================] - 2s 7ms/step - loss: 0.3608 - accuracy: 0.8432
Epoch 8/50
250/250 [==============================] - 2s 7ms/step - loss: 0.3795 - accuracy: 0.8376
Epoch 9/50
250/250 [==============================] - 2s 7ms/step - loss: 0.3325 - accuracy: 0.8608
Epoch 10/50
250/250 [==============================] - 2s 8ms/step - loss: 0.2548 - accuracy: 0.900

In [21]:
# モデルを評価する
score = model.evaluate(X_test, y_test)
print('loss = ', score[0])
print('accuracy = ', score[1])

84/84 [==============================] - 0s 4ms/step
loss =  0.8719884355862936
accuracy =  0.8571428656578064


In [26]:
pred = model.predict(X_test)
pred

array([[0.21265103, 0.21737385, 0.19056967, 0.20368071, 0.17572476],
       [0.21149427, 0.22560222, 0.19560033, 0.20728163, 0.16002157],
       [0.20762134, 0.23409617, 0.19431809, 0.22895154, 0.13501285],
       [0.20306325, 0.20978306, 0.20146748, 0.2009839 , 0.18470234],
       [0.19236596, 0.24008805, 0.18622461, 0.20778123, 0.17354016],
       [0.20259292, 0.20828623, 0.20178188, 0.20511134, 0.18222758],
       [0.20658545, 0.21623552, 0.19497135, 0.204269  , 0.1779386 ],
       [0.21649608, 0.20936944, 0.1891503 , 0.21043229, 0.17455184],
       [0.2080398 , 0.22117908, 0.19319314, 0.21270412, 0.16488375],
       [0.21042706, 0.20857693, 0.19216272, 0.2097585 , 0.1790748 ],
       [0.19982561, 0.20559318, 0.1973809 , 0.20654738, 0.19065295],
       [0.21254793, 0.23289423, 0.19172752, 0.19432507, 0.16850525],
       [0.19068293, 0.22588745, 0.19921035, 0.2351392 , 0.14908005],
       [0.20148061, 0.22002967, 0.19702975, 0.20316556, 0.17829435],
       [0.21761064, 0.20258208, 0.

### さらに精度を上げるために